In [16]:
import logging
import pandas as pd
import requests
import sqlite3
from pandas import json_normalize

# Configure logging to output to a file
logging.basicConfig(
    filename='extraction.log',
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
)


In [17]:
import logging
import pandas as pd 


def extract_csv(file_path: str):
    import logging
    import pandas as pd

    logging.info(f'Attempting to read CSV file {file_path}')
    try:
        # Try UTF-8 first
        df = pd.read_csv(file_path, encoding='utf-8')
        logging.info(f'Successfully read CSV file with UTF-8, rows={len(df)}')
        return df
    except UnicodeDecodeError:
        logging.warning('UTF-8 decoding failed, retrying with ISO-8859-1 encoding...')
        try:
            df = pd.read_csv(file_path, encoding='ISO-8859-1')
            logging.info(f'Successfully read CSV file with ISO-8859-1, rows={len(df)}')
            return df
        except Exception as e:
            logging.error(f'Failed to read CSV with fallback encoding: {e}')
            return None
    except FileNotFoundError:
        logging.error(f'Error: File not found at {file_path}')
        return None
    except Exception as e:
        logging.error(f'An unexpected error occurred while reading {file_path}: {e}')
        return None
    
def transform(df: pd.DataFrame):
    import logging
    import pandas as pd

    logging.info('Starting data transformation')
    try:
        # Example transformation: drop rows with any null values
        initial_rows = len(df)
        df_cleaned = df.dropna()
        final_rows = len(df_cleaned)
        logging.info(f'Dropped {initial_rows - final_rows} rows with null values')
        return df_cleaned
    except Exception as e:
        logging.error(f'Error during data transformation: {e}')
        return df


In [18]:
def run():
    import logging
    import pandas as pd
    import sqlite3

    logging.info('ETL process started')

    # Extraction
    file_path = 'C:\\Users\\novic\\OneDrive\\Desktop\\DE\\extraction\\dataset.csv'

    df = extract_csv(file_path)
    if df is None:
        logging.error('Extraction failed, terminating ETL process')
        return

    # Transformation
    df_transformed = transform(df)

    # Load to SQLite
    conn = sqlite3.connect('spotify.db')
    cur = conn.cursor()
    try:
        df_transformed.to_sql('catalog', conn, if_exists='replace', index=False)
        logging.info(f'Successfully loaded data into SQLite database, rows={len(df_transformed)}')
    except Exception as e:
        logging.error(f'Error loading data into SQLite: {e}')
    finally:
        conn.close()

    logging.info('ETL process completed successfully')
if __name__ == '__main__':
    run()

In [ ]:
# query to verify data in SQLite
def verify_data():
    import logging
    import sqlite3

    logging.info('Verifying data in SQLite database')
    conn = sqlite3.connect('spotify.db')
    cur = conn.cursor()
    try:
        row = pd.read_sql_query('SELECT * FROM catalog LIMIT 5;', conn)
        print(f'First 5 rows from catalog table:\n{row}')   
    except Exception as e:
        logging.error(f'Error verifying data in SQLite: {e}')
    finally:
        conn.close()
if __name__ == '__main__':
    verify_data()